# Deep Learning - Exercise 7
This lecture is focused on the more advanced examples of the RNN usage for text data anylysis.

We will deal with the sentiment analysis task using Twitter data.

You can download the dataset from [this link](https://github.com/MohamedAfham/Twitter-Sentiment-Analysis-Supervised-Learning/tree/master/Data)


[Open in Google colab](https://colab.research.google.com/github/rasvob/2020-21-ARD/blob/master/abd_07.ipynb)
[Download from Github](https://github.com/rasvob/2020-21-ARD/blob/master/abd_07.ipynb)

In [ ]:
import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import tensorflow as tf #use tensorflow 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import normalize # normalization of the matrix
import scipy
import pandas as pd

tf.version.VERSION

In [ ]:
import unicodedata, re, string
import nltk
from textblob import TextBlob

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

In [ ]:
nltk.download('punkt')

In [ ]:
df = pd.read_csv('https://github.com/jplatos/VSB-FEI-Deep-Learning/raw/main/datasets/train_tweets.csv')

In [ ]:
df.head()

# Let's take a look at the data

In [ ]:
df.shape

## We can see that the classification task is highly imbalanced, because we have only 2242 negative tweets compared with positive one

In [ ]:
sns.countplot(x='label', data=df)

In [ ]:
df.label.value_counts()

In [ ]:
df['length'] = df.tweet.apply(len)

### We can see that the sentences are of similar lengths

In [ ]:
sns.barplot(x='label', y='length', data = df)

# We can see that the text data are full of noise

- Social posts suffer the most from this effect
- The text is full of hashtags, emojis, @mentions and so on
- These parts usually don't influence the sentiment score by much
- Although most advanced models usually extract even this features because e.g. emojis can help you with the sarcasm understanding

In [ ]:
for x in df.loc[:10, 'tweet']:
    print(x)
    print('---------')

## Stemming
Stemming is the process of producing morphological variants of a root/base word. Stemming programs are commonly referred to as stemming algorithms or stemmers. A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve”,

## Lemmatization 
Lemmatization is the process of grouping together the different inflected forms of a word so they can be analysed as a single item. Lemmatization is similar to stemming but it brings context to the words. So it links words with similar meaning to one word.

Examples of lemmatization:

- rocks : rock
- corpora : corpus
- better : good

## Both techiques can be used in the preprocessing pipeline
You have to decide if it is beneficial to you, because this steps leads to some generalization of the data by itself. You will definitely lose some pieces of the information. If you use some form of embedding like Word2Vec or Glove, it is better to skip this steps because the embedding vocabulary skipped it as well.

In [ ]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_numbers(words):
    """Remove all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        new_word = re.sub("\d+", "", word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
# words = remove_punctuation(words)
    words = remove_numbers(words)
#    words = remove_stopwords(words)
    return words

def form_sentence(tweet):
    tweet_blob = TextBlob(tweet)
    return tweet_blob.words

# Tokenize sentences and remove puncuation by TextBlob library

In [ ]:
df['Words'] = df['tweet'].apply(form_sentence)

In [ ]:
df.head()

# Normalize sentences 
- We want only ascii, lowercase and no numbers

## You can experiments with different preprocess steps!

In [ ]:
df['Words_normalized'] = df['Words'].apply(normalize)

In [ ]:
df.head()

## Remove the 'user' word from tweets

In [ ]:
df['Words_normalized_no_user'] = df['Words_normalized'].apply(lambda x: [y for y in x if 'user' not in y])

In [ ]:
df.head()

## We can see that no pre-processing is ideal and we have to fix some issues by ourselves
- e.g. n't splitting

In [ ]:
print(df.tweet.iloc[1])
print(df.Words_normalized_no_user.iloc[1])

In [ ]:
def fix_nt(words):
    st_res = []
    for i in range(0, len(words) - 1):
        if words[i+1] == "n't" or words[i+1] == "nt":
            st_res.append(words[i]+("n't"))
        else:
            if words[i] != "n't" and words[i] != "nt":
                st_res.append(words[i])
    return st_res

In [ ]:
df['Words_normalized_no_user_fixed'] = df['Words_normalized_no_user'].apply(fix_nt)

## The issue is now fixed

In [ ]:
print(df.tweet.iloc[1])
print(df.Words_normalized_no_user.iloc[1])
print(df.Words_normalized_no_user_fixed.iloc[1])

In [ ]:
df['Clean_text'] = df['Words_normalized_no_user_fixed'].apply(lambda x: " ".join(x))

In [ ]:
df.head()

# Let's take a look at the most common words in corpus

In [ ]:
import itertools

In [ ]:
all_words = list(itertools.chain(*df.Words_normalized_no_user_fixed))

In [ ]:
dist = nltk.FreqDist(all_words)

In [ ]:
dist

### How many words do we have?

In [ ]:
len(dist)

### How long is the longest tweet?

In [ ]:
max(df.Words_normalized_no_user_fixed.apply(len))

# We will use the TextVectorization layer for creating vector model from our text data
For those of you who are interested in the topic there is very good [article on Medium](https://towardsdatascience.com/you-should-try-the-new-tensorflows-textvectorization-layer-a80b3c6b00ee) about the layer and its parameters.

There is of course a [documentation page](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) about the layer.


In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers import TextVectorization

In [ ]:
embedding_dim = 128 # Dimension of embedded representation - this is already part of latent space, there is captured some dependecy among words, we are learning this vectors in ANN
vocab_size = 10000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing - words in vectorited representation are independent

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(df.Clean_text.values)



### We will split our dataset to train and test parts with stratification

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.Clean_text, df.label, test_size=0.20, random_state=13, stratify=df.label)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=13, stratify=y_train)

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
print('Train')
print(y_train.value_counts())
print('Test')
print(y_test.value_counts())

In [ ]:
print('Vocabulary example: ', vect_layer.get_vocabulary()[:10])
print('Vocabulary shape: ', len(vect_layer.get_vocabulary()))

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = keras.layers.LSTM(64, activation='relu', return_sequences=True)(emb)
x = keras.layers.GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

model.compile(optimizer='rmsprop', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])


In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 128
epochs = 5
history = model.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
show_history(history)

In [ ]:
y_test_loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test).ravel()

#### Sigmoid function gives us real number in range <0, 1>.

#### We need to map this valus to classes

In [ ]:
y_pred

In [ ]:
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]

# We can see that accuracy is not the best metric in the imbalanced situation - why?
There are many more metrics we can use and one of the most common in this situation is the F1 Score, see [this](https://en.wikipedia.org/wiki/F-score) and [this](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/) for more info

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_pred)

In [ ]:
f1_score(y_true=y_test, y_pred=y_pred)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
print(confusion_matrix(y_true=y_test, y_pred=y_pred))

# We don't have to train our own embedding
There are multiple embeddings available online which were trained on very large corpuses e.g. Wikipedia. Good examples are Word2Vec, Glove or FastText. These embeddings contains fixed length vectors for words in the vocabulary.

We will use GloVe embedding with 50 dimensional embedding vectors. For more details see [this](https://nlp.stanford.edu/projects/glove/).
You can download zip with vectors from [http://nlp.stanford.edu/data/glove.6B.zip](http://nlp.stanford.edu/data/glove.6B.zip) ~ 800 MB

#### Beware that the original text corpus was more general than the specific social media text data, so if you deal with very specific domains it may be beneficial to train your own embedding or at least fine tune existing one.

# We need to download the embedding files

~~~
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
~~~

The file glove.6B.50d.txt is also available in [Github](https://github.com/jplatos/VSB-FEI-Deep-Learning/raw/main/datasets/glove.6B.50d.zip) repository.

In [ ]:
!wget https://github.com/jplatos/VSB-FEI-Deep-Learning/raw/main/datasets/glove.6B.50d.zip
!unzip -q glove.6B.50d.zip

# First we need to load the file to memory and create embedding dictionary

In [ ]:
path_to_glove_file = 'glove.6B.50d.txt'

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
embeddings_index['analysis']

## We need to get the voacabulary from the Vectorizer and the integer indexes

In [ ]:
embedding_dim = 50 # Dimension of embedded representation - this is already part of latent space, there is captured some dependecy among words, we are learning this vectors in ANN
vocab_size = 10000 # Number of unique tokens in vocabulary
sequence_length = 20 # Output dimension after vectorizing - words in vectorited representation are independent

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(df.Clean_text.values)

In [ ]:
voc = vect_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
voc[:10]

In [ ]:
word_index['the']

In [ ]:
embeddings_index['the']

In [ ]:
num_tokens = len(voc) + 2
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
embedding_matrix[2]

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = keras.layers.LSTM(64, activation='relu', return_sequences=True)(emb)
x = keras.layers.GRU(64, activation='relu', return_sequences=False)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

model.compile(optimizer='rmsprop', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])


In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 128
epochs = 5
history = model.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
show_history(history)

In [ ]:
y_pred = model.predict(X_test).ravel()
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
print(f'Accuracy: {accuracy_score(y_true=y_test, y_pred=y_pred)}')
print(f'F1 Score: {f1_score(y_true=y_test, y_pred=y_pred)}')
print(confusion_matrix(y_true=y_test, y_pred=y_pred))

# Task for the lecture
 - Try to create your own architecture
 - Experiment a little - try different batch sizes, optimimizers, etc
 - Send me a link to the Colab notebook with results and description of what you did and your final solution!
 
# There is a competition for bonus points this week!
- Everyone who will send me a correct solution will be included in the F1 - Score toplist
- Deadline for the competition submission is Monday 14th at 23:59
- The toplist will be publicly available on Wednesday
- There is no limitation in used layers (LSTM, CNN, ...), optimizers, etc. 
- You can use any model architecture from the internet including transfer learning,
- The only limitation is that the model has to be trained/fine-tuned on Colab/Kaggle/Your machine so online sentiment scoring services are forbidden!

## The winner with the best F1 - Score on test set will be awarded with 5 bonus points
- The test set is the same as we used in the lecture